In [8]:
# Run a Scikit-Learn Random Forest Regressor on the ABIDE FreeSurfer meshes,
# to predict the local gyrification index at each vertex.

import numpy as np
import pandas as pd
import psutil
import sys
import os
import time
from datetime import timedelta
import psutil
from sys import getsizeof

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from meshlearn.training_data import TrainingData, get_valid_mesh_desc_file_pairs_reconall


# Prepare to access the data, stored in Google Drive of the 
# Colab user (same Google Account).
from google.colab import drive
drive.mount('/content/drive') # This will prompt for authorization.

#!ls "/content/drive/My Drive" # Show data in Google Drive, to be sure it worked.




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 abide_freesurfer_lgi_2persite
 abide_freesurfer_lgi_predict
'Colab Notebooks'
'Copy of Teampaper_BrainMap_for_export.gdoc'
 Export_Copy_of_Teampaper_BrainMap_do_not_edit_online.gdoc
 Export_Teampaper_BrainMap_Export_noen_nocomments.gdoc
'Power in sMRI.gslides'
 Teampaper_BrainMap_Export_noen_nocomments.gdoc


In [9]:

# Install meshlearn. The '-q' is for quiet, to prevent long output in notebook.
!pip install -q git+https://github.com/dfsp-spirit/meshlearn.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/dfsp-spirit/meshlearn.git to /tmp/pip-req-build-y6vnu5t6
  Running command git clone -q https://github.com/dfsp-spirit/meshlearn.git /tmp/pip-req-build-y6vnu5t6


In [ ]:

"""
Train and evaluate an lGI prediction model.
"""

### Settings

data_dir="/content/drive/My Drive/abide_freesurfer_lgi_2persite"
neigh_count = 300  # Number of vertices to consider at max in the edge neighborhoods for Euclidean dist
neigh_radius = 10  #  Radius for sphere for Euclidean dist, in spatial units of mesh (e.g., mm).
load_max = 40_000  # Number of samples to load. Set to 0 for all data in the files discovered in the data_dir.
verbose = True

### End of Settings

mesh_neighborhood_count = neigh_count # How many vertices in the edge neighborhood do we consider (the 'local' neighbors from which we learn).
mesh_neighborhood_radius = neigh_radius

num_neighborhoods_to_load = None if int(load_max) == 0 else int(load_max)

print("---Train and evaluate an lGI prediction model---")
if verbose:
    print("Verbosity turned on.")

print(f"Using data directory '{data_dir}', observations to load limit is set to: {num_neighborhoods_to_load}.")
print(f"Using neighborhood radius {mesh_neighborhood_radius} and keeping {mesh_neighborhood_count} vertices per neighborhood.")

if num_neighborhoods_to_load is not None:
    # Estimate total dataset size in RAM early to prevent crashing later, if possible.
    ds_estimated_num_values_per_neighborhood = 6 * mesh_neighborhood_count + 1
    ds_estimated_num_neighborhoods = num_neighborhoods_to_load
    # try to allocate, will err if too little RAM.
    print(f"RAM available is about {int(psutil.virtual_memory().available / 1024. / 1024.)} MB")
    ds_dummy = np.empty((ds_estimated_num_neighborhoods, ds_estimated_num_values_per_neighborhood))
    ds_estimated_full_data_size_bytes = getsizeof(ds_dummy)
    ds_dummy = None
    ds_estimated_full_data_size_MB = ds_estimated_full_data_size_bytes / 1024. / 1024.
    print(f"Estimated dataset size in RAM will be {int(ds_estimated_full_data_size_MB)} MB.")



discover_start = time.time()
mesh_files, desc_files, cortex_files, val_subjects = get_valid_mesh_desc_file_pairs_reconall(data_dir)
discover_end = time.time()
discover_execution_time = discover_end - discover_start
print(f"=== Discovering data files done, it took: {timedelta(seconds=discover_execution_time)} ===")

### Decide which files are used as training, validation and test data. ###
input_file_dict = dict(zip(mesh_files, desc_files))

if verbose:
    print(f"Discovered {len(input_file_dict)} valid pairs of input mesh and descriptor files.")

if num_neighborhoods_to_load is None:
    print(f"Will load all data from the {len(input_file_dict)} files.")
else:
    print(f"Will load {num_neighborhoods_to_load} samples in total from the {len(input_file_dict)} files.")

load_start = time.time()
tdl = TrainingData(neighborhood_radius=mesh_neighborhood_radius, num_neighbors=mesh_neighborhood_count)
dataset, col_names = tdl.load_raw_data(input_file_dict, num_samples_to_load=num_neighborhoods_to_load)
load_end = time.time()
load_execution_time = load_end - load_start
print(f"=== Loading data files done, it took: {timedelta(seconds=load_execution_time)} ===")

assert isinstance(dataset, pd.DataFrame)

print("Separating observations and labels...")

nc = len(dataset.columns)
X = dataset.iloc[:, 0:(nc-1)].values
y = dataset.iloc[:, (nc-1)].values

print("Splitting data into train and test sets...")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(f"Created training data set with shape {X_train.shape} and testing data set with shape {X_test.shape}.")
print(f"The label arrays have shape {y_train.shape} for the training data and  {y_test.shape} for the testing data.")


print("Scaling...")

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



n_estimators = 100


fit_start = time.time()
print(f"Fitting with RandomForestRegressor with {n_estimators} estimators. (Started at {fit_start}.)")

regressor = RandomForestRegressor(n_estimators=n_estimators, random_state=0, n_jobs=-1)
regressor.fit(X_train, y_train)

fit_end = time.time()
fit_execution_time = fit_end - fit_start

print(f"===Fitting done, it took: {timedelta(seconds=fit_execution_time)} ===")
print(f"Using trained model to predict for test data set with shape {X_test.shape}.")

y_pred = regressor.predict(X_test)


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# Evaluate feature importance
importances = regressor.feature_importances_

feature_names = col_names[:-1]

print(f"=== Evaluating Feature importance ===")
print(f"Feature names: {feature_names}")
print(f"Feature impor: {importances}")

do_plot = False
if do_plot:
    std = np.std([tree.feature_importances_ for tree in regressor.estimators_], axis=0)
    forest_importances = pd.Series(importances, index=feature_names)
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=std, ax=ax)
    ax.set_title("Feature importances using MDI")
    ax.set_ylabel("Mean decrease in impurity")
    fig.tight_layout()




---Train and evaluate an lGI prediction model---
Verbosity turned on.
Using data directory '/content/drive/My Drive/abide_freesurfer_lgi_2persite', observations to load limit is set to: 40000.
Using neighborhood radius 10 and keeping 300 vertices per neighborhood.
RAM available is about 11698 MB
Estimated dataset size in RAM will be 549 MB.
Using subjects list containing 32 subjects. Loading them from recon-all output dir '/content/drive/My Drive/abide_freesurfer_lgi_2persite'.
Discovering surface 'pial', descriptor 'pial_lgi' for 2 hemis: ['lh', 'rh'].
Not discovering cortex labels.
Out of 64 subject hemispheres (32 subjects), 63 had the requested surface and descriptor files.
=== Discovering data files done, it took: 0:00:15.269974 ===
Discovered 63 valid pairs of input mesh and descriptor files.
Will load 40000 samples in total from the 63 files.
[load] Loading data.
[load] * Loading mesh file '/content/drive/My Drive/abide_freesurfer_lgi_2persite/Caltech_0051456/surf/lh.pial' and d